# HW3: 新規ユーザーの初日の行動と翌日起動の関係を調べる

＜お題＞
あるキュレーションアプリのユーザーデータのテーブルuser_data.user_masterとstat.user_article_statを使い、2016年1月1日入会ユーザーについて、各ユーザーの入会後24時間以内行動データ（以下のような表）と次の24時間から48時間でアプリを起動したかどうかを作成してください。

|user_id|user_login|app_open|topic_follow|article_show|article_read|翌日起動したか|
|:-----:|:--------:|:------:|:----------:|:-----------:|:----------:|:----------:|
|001|1|1|0|1|0|1|

## 質問
ダウンロード翌日（ダウンロードしてから24時間後~48時間後)起動する人とそうでない人について、初日にとった以下の行動の回数で大きな差異が見られますか？

もし差異があるとしたら、どの行動ですか？

* app_open（アプリの起動）
* topic_follow　（テーマをフォローした）
* article_show　（記事タイトルを見た）
* article_read　（記事本文を閲覧した）


In [1]:
%matplotlib inline
import pandas as pd
import pandas.io.sql as psql
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

import psycopg2

from __future__ import print_function

## データベースへのアクセスを確立する

In [2]:
db_params = {
    'dbname': 'XXXXXXXX',
    'user': 'XXXXXXX',
    'password': 'XXXXXXXXX',
    'host': "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX",
    'port': XXXX
}

con = psycopg2.connect(**db_params)


## DBからデータを取得する

ユーザー毎に、アプリ起動回数、テーマフォロー数、記事タイトル表示数、記事本文閲覧数を集計しながら取得する。

In [3]:
my_sql_hw="""
WITH
target_user as(
select*
from user_data.user_master
WHERE created_at + interval '9hours' >='2016-01-01 00:00:00'
and created_at + interval '9hours' <'2016-01-02 00:00:00'),

first_day_user as(
select
a.user_id,
sum(case when b.behave_type ='app_open' then 1 else 0 end) as app_open_count,
sum(case when b.behave_type ='topic_follow' then 1 else 0 end) as topic_follow_count,
sum(case when b.behave_type ='article_show' then 1 else 0 end) as article_show_count,
sum(case when b.behave_type ='article_read' then 1 else 0 end) as article_read_count
from target_user as a
inner join
stat.user_article_stat as b
on a.user_id = b.user_id
where b.created_at >= a.created_at
AND b.created_at < a.created_at + interval '24hours'
GROUP BY a.user_id
),

appopen_nextday as(
SELECT
target_user.user_id,
SUM(CASE WHEN stat.user_article_stat.behave_type ='app_open' THEN 1 ELSE 0 END) AS appopen_nextday_count
FROM target_user
INNER JOIN
stat.user_article_stat
on target_user.user_id = stat.user_article_stat.user_id
WHERE stat.user_article_stat.created_at >= target_user.created_at + interval '24hours'
AND stat.user_article_stat.created_at <= target_user.created_at + interval '48hours'
AND stat.user_article_stat.behave_type ='app_open'
GROUP BY target_user.user_id
)

SELECT 
c.user_id,
app_open_count,
topic_follow_count,
article_show_count,
article_read_count,
appopen_nextday_count,
CASE WHEN d.appopen_nextday_count is NULL THEN 0 ELSE 1 END AS appopen_nextday_xx
FROM first_day_user as c
LEFT OUTER JOIN appopen_nextday as d
ON c.user_id = d.user_id
"""

user_article_data = psql.read_sql(con=con, sql=my_sql_hw)
user_article_data.head()

,user_id,app_open_count,topic_follow_count,article_show_count,article_read_count,appopen_nextday_count,appopen_nextday_xx
0,758530,0,0,0,0,NaN,0
1,758454,1,0,27,0,3.0,1
2,758459,4,0,12,3,5.0,1
3,758470,5,0,266,14,NaN,0
4,758481,21,0,1864,65,4.0,1


In [4]:
user_article_data.shape

(94, 7)

## 結果の確認
ダウンロード翌日に起動したユーザーとしなかったユーザーそれぞれで、各カラムの平均をとり、アプリ起動ユーザー群で特に割合の高い項目を探す。

In [5]:
appopen_y_data = user_article_data[user_article_data["appopen_nextday_xx"] == 1]
appopen_n_data = user_article_data[user_article_data["appopen_nextday_xx"] == 0]

#それぞれaverageを出す
appopen_y_appopen = appopen_y_data["app_open_count"].mean()
appopen_n_appopen = appopen_n_data["app_open_count"].mean()

print ("アプリの起動:",appopen_y_appopen / appopen_n_appopen)

appopen_y_topicfollow = appopen_y_data["topic_follow_count"].mean()
appopen_n_topicfollow = appopen_n_data["topic_follow_count"].mean()

appopen_y_artshow = appopen_y_data["article_show_count"].mean() 
appopen_n_artshow = appopen_n_data["article_show_count"].mean()

print("記事タイトルを見た:",appopen_y_artshow/appopen_n_artshow)

appopen_y_artread = appopen_y_data["article_read_count"].mean()
appopen_n_artread = appopen_n_data["article_read_count"].mean()

print("記事本文を閲覧した=最も大きな差異:",appopen_y_artread/appopen_n_artread)

アプリの起動: 2.9345776772247363
記事タイトルを見た: 6.26337672274068
記事本文を閲覧した=最も大きな差異: 18.371448371448373
